# Elasticsearh - usign REST interface

in these examples we use curl but you can use wget, PUT, GET, DELETE where supported

### Check we can connect to the cluster

In [13]:
# define our address so we dont have to type it all the time
address='atlas-kibana.mwt2.org:9200'

! curl "$address/_cluster/health"

{"cluster_name":"atlas","status":"green","timed_out":false,"number_of_nodes":10,"number_of_data_nodes":5,"active_primary_shards":13347,"active_shards":18485,"relocating_shards":0,"initializing_shards":0,"unassigned_shards":0,"delayed_unassigned_shards":0,"number_of_pending_tasks":6,"number_of_in_flight_fetch":0,"task_max_waiting_in_queue_millis":900,"active_shards_percent_as_number":100.0}

### Index one document we have in a json file

In [22]:
! curl -XPOST "$address/my_events/event"  -d @OneDoc.json

{"_index":"my_events","_type":"event","_id":"AV9pfAtZP9IDFhhRXhxR","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"created":true}

### BULK Index the documents we have in a json file

In [41]:
! curl -XPOST "$address/my_events/_bulk" --data-binary "@ExampleDocs.json"

{"took":18,"errors":false,"items":[{"index":{"_index":"my_events","_type":"event","_id":"AV9pqYhNP9IDFhhRu6Gp","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"created":true,"status":201}},{"index":{"_index":"my_events","_type":"event","_id":"AV9pqYhNP9IDFhhRu6Gq","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"created":true,"status":201}},{"index":{"_index":"my_events","_type":"event","_id":"AV9pqYhNP9IDFhhRu6Gr","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"created":true,"status":201}}]}

### Get information on what and index settings and docs we have in it

In [45]:
! curl -XGET "$address/my_events/_settings"

# number of documents
! curl -XGET "$address/my_events/event/_count"  

{"my_events":{"settings":{"index":{"creation_date":"1509303043133","number_of_shards":"5","number_of_replicas":"1","uuid":"38nca8giTCKfkjujdy1RwA","version":{"created":"5060299"},"provided_name":"my_events"}}}}

### Check that the data has been correctly mapped

In [46]:
! curl -XGET "$address/my_events/_mappings"

{"my_events":{"mappings":{"event":{"properties":{"description":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"eventnr":{"type":"long"},"timestamp":{"type":"long"},"tracks":{"properties":{"chi2":{"type":"float"},"kind":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"pT":{"type":"float"}}}}}}}}

### Delete newly created index

In [48]:
!curl -XDELETE "$address/my_events"

{"acknowledged":true}

### create a template for our data

In [57]:
! curl -XPOST "$address/_template/my_events" -d @ExampleTemplate.json


{"acknowledged":true}

### index the data again

In [58]:
! curl -XPOST "$address/my_events/_bulk" --data-binary "@ExampleDocs.json"

{"took":6357,"errors":false,"items":[{"index":{"_index":"my_events","_type":"event","_id":"AV9ptynNP9IDFhhRvPGV","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"created":true,"status":201}},{"index":{"_index":"my_events","_type":"event","_id":"AV9ptynNP9IDFhhRvPGW","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"created":true,"status":201}},{"index":{"_index":"my_events","_type":"event","_id":"AV9ptynNP9IDFhhRvPGX","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"created":true,"status":201}}]}

### Check mapping now 

In [59]:
! curl -XGET "$address/my_events/_mappings"

{"my_events":{"mappings":{"event":{"properties":{"description":{"type":"text"},"energy":{"type":"float"},"eventnr":{"type":"long"},"timestamp":{"type":"date"},"tracks":{"properties":{"chi2":{"type":"float"},"kind":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"pT":{"type":"float"}}}}}}}}

### Search for a document

In [64]:
! curl -XGET "$address/my_events/event/_search?q=eventnr:>1"

{"took":158,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":3,"max_score":1.0,"hits":[{"_index":"my_events","_type":"event","_id":"AV9ptynNP9IDFhhRvPGV","_score":1.0,"_source":{ "eventnr":1231455, "description":"my higgs candidate1", "timestamp":1509301292000, "tracks":[ { "kind":"InDet", "pT":1324.123, "chi2":1.234 }, { "kind":"MuonSeg", "pT":1324.123, "chi2":1.234 } ] }},{"_index":"my_events","_type":"event","_id":"AV9ptynNP9IDFhhRvPGW","_score":1.0,"_source":{ "eventnr":2222222, "description":"my higgs candidate2", "timestamp":1509301293000, "tracks":[ { "kind":"InDet", "pT":1324.123, "chi2":1.234 }, { "kind":"MuonSeg", "pT":1324.123, "chi2":1.234 } ] }},{"_index":"my_events","_type":"event","_id":"AV9ptynNP9IDFhhRvPGX","_score":1.0,"_source":{ "eventnr":3333333, "description":"my higgs candidate3", "timestamp":1509301294000, "tracks":[ { "kind":"InDet", "pT":1324.123, "chi2":1.234 }, { "kind":"MuonSeg", "pT":1324.123, "chi2":1.234 } ] }

### Perform aggregated search

In [71]:
! curl -XGET "$address/my_events/event/_count"  -d '{ "query" : { "term" : { "user" : "kimchy" }  } }'

{"count":3,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}

### Clean up

In [ ]:
!curl -XDELETE "$address/my_events"